In [3]:
import pandas as pd
import requests
import json

In [ ]:
# r = requests.get(f'https://www.immoweb.be/en/search-results/house-and-apartment/for-rent?countries=BE&page=1&orderBy=newest')

In [ ]:
# with open("data/response.json", "w") as f:
#     f.write(str(json.dumps(r.json()["results"])))

In [23]:
properties = {}
n = 1
while True:
    try:
        r = requests.get(f'https://www.immoweb.be/en/search-results/house-and-apartment/for-rent?countries=BE&page={n}&orderBy=newest')
        list_ads = r.json()["results"]
        for ad in list_ads:

            # rebuild ad url
            property_type = ad["property"]["type"]
            locality = ad["property"]["location"]["locality"]
            postalcode = ad["property"]["location"]["postalCode"]
            id = ad["id"]
            url = f"https://www.immoweb.be/en/classified/{property_type}/for-rent/{locality}/{postalcode}/{id}"
            
            # avoid duplicates
            if url in properties:
                continue

            # add property to properties
            property = {
                "id": ad.get("id"),
                "type": ad.get("property", {}).get("type").lower(),
                "subtype": ad.get("property", {}).get("subtype").lower(),
                "number of bedrooms": ad.get("property", {}).get("bedroomCount"),
                "country": ad.get("property", {}).get("location", {}).get("country"),
                "region": ad.get("property", {}).get("location", {}).get("region"),
                "province": ad.get("property", {}).get("location", {}).get("province"),
                "district": ad.get("property", {}).get("location", {}).get("district"),
                "locality": ad.get("property", {}).get("location", {}).get("locality"),
                "postal_code": ad.get("property", {}).get("location", {}).get("postalCode"),
                "street": ad.get("property", {}).get("location", {}).get("street"),
                "number": ad.get("property", {}).get("location", {}).get("number"),
                "floor": ad.get("property", {}).get("location", {}).get("floor"),
                "latitude": ad.get("property", {}).get("location", {}).get("latitude"),
                "longitude": ad.get("property", {}).get("location", {}).get("longitude"),
                "net_habitable_surface": ad.get("property", {}).get("netHabitableSurface"),
                "monthly_rental_price": ad.get("transaction", {}).get("rental", {}).get("monthlyRentalPrice"),
                "monthly_rental_costs": ad.get("transaction", {}).get("rental", {}).get("monthlyRentalCosts"),
            }
            properties[url] = property
            
            # create dataframe from properties and save as csv file
            df = pd.DataFrame.from_dict(properties, orient='index')
            df.to_csv('data/save_df.csv')
        
        # go to next page
        n += 1

    except:
        break

    # test on 10 pages
    # if n == 11:
    #     break

pd.read_csv('data/save_df.csv').shape

(9972, 19)

In [25]:
pd.read_csv('data/save_df.csv').head()

,Unnamed: 0,id,type,subtype,number of bedrooms,country,region,province,district,locality,postal_code,street,number,floor,latitude,longitude,net_habitable_surface,monthly_rental_price,monthly_rental_costs
0,https://www.immoweb.be/en/classified/APARTMENT...,10005791,apartment,apartment,1.0,Belgium,Flanders,Antwerp,Antwerp,Antwerpen,2000,Tolstraat,8,2.0,51.209975,4.398165,100.0,1100.0,95.0
1,https://www.immoweb.be/en/classified/HOUSE/for...,10005790,house,house,3.0,Belgium,Flanders,West Flanders,Brugge,Brugge,8000,Haarakkerstraat,24,NaN,51.219858,3.232593,112.0,850.0,NaN
2,https://www.immoweb.be/en/classified/APARTMENT...,10005788,apartment,apartment,1.0,Belgium,Brussels,Brussels,Brussels,Saint-Josse-ten-Noode,1210,Rue Hydraulique,26,1.0,50.848119,4.372161,25.0,800.0,NaN
3,https://www.immoweb.be/en/classified/APARTMENT...,10005787,apartment,apartment,1.0,Belgium,Wallonie,Liège,Liège,Liège,4000,NaN,NaN,2.0,NaN,NaN,48.0,650.0,30.0
4,https://www.immoweb.be/en/classified/HOUSE/for...,10005784,house,house,3.0,Belgium,Wallonie,Walloon Brabant,Nivelles,Orp-Jauche,1350,Rue De La Frête,6,NaN,50.656973,4.930566,180.0,1050.0,NaN


In [26]:
pd.read_csv('data/save_df.csv').isnull().sum()

Unnamed: 0                  0
id                          0
type                        0
subtype                     0
number of bedrooms          2
country                     0
region                      1
province                    1
district                    1
locality                    0
postal_code                 0
street                   1968
number                   2162
floor                    1804
latitude                 1976
longitude                1976
net_habitable_surface    2295
monthly_rental_price        2
monthly_rental_costs     4466
dtype: int64